In [15]:
username       = ""
password       = ""
profile_type   = 'okr-sprout'

In [22]:
import requests
import urllib

url = 'https://demo.grnry.io/auth/realms/grnry/protocol/openid-connect/token'

payload = f'grant_type=password&username={urllib.parse.quote_plus(username)}&password={urllib.parse.quote_plus(password)}&client_id=profile-api'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request('POST', url, headers=headers, data = payload)

token = response.json()['access_token']

In [23]:
responses = []
for correlation_id in ['CS','product','growth']:
  url = f"https://demo.grnry.io/profiles/okr-sprout/{correlation_id}?fragments="
  headers = {
    'Authorization': f'Bearer {token}'}
  response = requests.request("GET", url, headers=headers)
  responses.append(response.json())



In [24]:
import pandas as pd

profiles = [ {'circle':p['correlationId'],**p['jsonPayload']} for p in responses]

for i in range(len(profiles)):
    profiles[i].pop('_id',None)
circles = pd.json_normalize(profiles,max_level=2)
circles = circles.melt(id_vars=['circle'])
values = pd.json_normalize(circles['value'],max_level=0)
values = values.applymap(lambda x: x.get('_latest').get('_v') if isinstance(x, dict) else '0')
okr_progress = pd.concat([circles[['circle','variable']],values],axis=1)
okr_progress


,circle,variable,numberOfKR,progressOfKR
0,CS,2020.11.26,10,75
1,product,2020.11.26,10,165
2,growth,2020.11.26,9,0
3,CS,2020.11.27,10,75
4,product,2020.11.27,10,165
5,growth,2020.11.27,9,0
6,CS,2020.11.28,10,75
7,product,2020.11.28,10,165
8,growth,2020.11.28,9,0
9,CS,2020.11.29,10,75


In [25]:
import numpy as np

values = okr_progress['progressOfKR'].astype(int)
nofokr = okr_progress['numberOfKR'].astype(int)
circle = okr_progress['circle']
dates = pd.to_datetime(okr_progress['variable'])

df=pd.DataFrame({'A': np.array(values/nofokr),
              'Circle': pd.Categorical(circle)},index=dates)
df=df.pivot(columns='Circle',values='A')

In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import datetime

fig = go.Figure()
fig = make_subplots(rows=2,cols=1)

days_since_trimester_start=max(df.index)-pd.to_datetime('2020-10-01')
progress_per_day=df.iloc[[-1]]/days_since_trimester_start.days
remaining_days=100/progress_per_day

for idx,c in enumerate(list(df)):
  if remaining_days.iat[0,idx]==float("inf"):
    eta=str('never')
  else:
    eta=datetime.timedelta(days=remaining_days.iat[0,idx])+pd.to_datetime('2020-10-01')
    eta=eta.strftime("%Y-%m-%d")
  
  fig.add_trace(go.Indicator(
      mode = "gauge+number",
      value = df[c].max()/100,
      domain = {'row': 0, 'column': idx},
      title = {'text': f"Progress {c}<br><span style='font-size:0.7em;color:gray'>Expected Completion Date: {eta}</span><br>"},
      gauge = {'axis': {'range': [None,1],
                        'tickformat':'%.0f%%'},
               'steps': [{'range':[0,0.4],'color':"mistyrose"},{'range':[0.4,0.7],'color':"lightyellow"},{'range':[0.7,1],'color':"honeydew"}],
               'threshold':{'line':{'color':"green",'width':4},'thickness':1,'value':0.70},
               'bar':{'color':"darkslategrey"}},
      number = {'valueformat':'%.0f%%'}))
  fig.add_trace(go.Scatter(x=df.index,y=df[c]/100,name=c,mode="lines"),row=2,col=1)

fig.update_layout(
    grid={'rows':2,'columns':3,'pattern':'independent'})

  

fig.update_yaxes(title_text="Progress",range=[0,1],tickformat='%.0f%%')
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.19,
    xanchor="left",
    x=0.01
))
#fig.update_layout(title_text="OKR Pogress")
fig.show()




In [11]:
df

Circle,CS,growth,product
variable,,,
2020-11-26,7.5,0.0,16.5
2020-11-27,7.5,0.0,16.5
2020-11-28,7.5,0.0,16.5
2020-11-29,7.5,0.0,16.5
2020-11-30,7.5,0.0,16.5
2020-12-01,7.5,0.0,16.5
2020-12-02,7.5,0.0,16.5
2020-12-03,7.5,0.0,16.5
2020-12-04,7.5,0.0,16.5
